In [1]:
import os
import yaml
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import spatial
import squidpy as sq
from sklearn.decomposition import non_negative_factorization

In [2]:
selected_K = 7
density_threshold = 0.1

input_directory = 'results'
adata_dir = 'adata'
adata_file = 'adata_full.h5ad'
adata_output_directory = 'adata_env'

program_names = ['env_gray_matter','env_hypoxic','env_white_matter','env_cellular_cancer','env_vasculature',\
                 'env_astro_inflammatory','env_MT-RPL']

In [3]:
# prepare
patients = [f[5:-5] for f in os.listdir(adata_dir)  if '.' != f[0]] 
density_threshold_str = ('%.2f' % density_threshold).replace('.', '_')
if not os.path.exists(adata_output_directory):
    os.mkdir(adata_output_directory)
    
k_filename = os.path.join(input_directory,'cnmf_run.k_selection_stats.df.npz')
with np.load(k_filename, allow_pickle=True) as f:
    k_obj = pd.DataFrame(**f)

In [4]:
# get usage scores
adata = sc.read(adata_file)
    
rf_usages = pd.read_csv(os.path.join(input_directory, 'cnmf_run.usages.k_%d.dt_%s.consensus.txt'\
                                       %(selected_K, density_threshold_str)), sep='\t', index_col=0)
rf_usages.columns = program_names
norm_usages = rf_usages.div(rf_usages.sum(axis=1), axis=0)
    
for col in norm_usages:
    adata.obs[col] = norm_usages[col]
                                 
# save adata for all patients
saved_adata = os.path.join('adata_env.h5ad')
adata.write(saved_adata)

In [5]:
# save adata for patients separately
for patient in patients:
    adata_patient = sc.read(os.path.join(adata_dir,'adata%s.h5ad'%patient))
    
    usages_patient = adata[adata.obs['patient']==patient,:].obs[program_names]
    usages_patient.index = [index.split('-')[0]+'-1' for index in usages_patient.index]
    
    for col in usages_patient:
        adata_patient.obs[col] = usages_patient[col]
        
    # save
    saved_adata = os.path.join(adata_output_directory, 'adata%s.h5ad'%patient)
    adata_patient.write(saved_adata)